In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm
from collections import Counter, defaultdict
import pickle
import csv
import argparse

In [2]:
df_raw = pd.read_csv('./t1_m10_raw.csv', sep='\t', dtype = {'Relation_id': str})

In [3]:
df_raw

,Subject,Relation_id,Relation_choice,Object,Md5,Date,Topic,Nday
0,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0
1,US,057,Sign formal agreement,Iran,0b6cd037f3fc22ed82e0c147839ba9f9,20150219,15,0
2,United States,051,Praise or endorse,Israel,0e3390e4d38e679fc25d764528a781e4,20150219,-1,0
3,United States,111,Criticize or denounce,Islam,0b79f78beb4ffba834ab552c7ef020ea,20150219,-1,0
4,Hamas,1831,Carry out suicide bombing,Israel,859fc96157484e73268e105e6222ce53,20150219,852,0
...,...,...,...,...,...,...,...,...
531286,Russia,061,Cooperate economically,Ukraine,3f48fb321a6a8e00f5f8a2c1fd756b87,20220317,1203,2583
531287,Ukraine,080,Yield or concede,Israel,33469c03a101913442242e89f18899ce,20220317,28,2583
531288,Israel,061,Cooperate economically,Ukraine,9e24beca44f3a9d8a966625f97b55814,20220317,28,2583
531289,Ukraine,194,Fight with artillery and tanks,Russia,cac2d7644682d7109ca0d2d7a5aac8d5,20220317,24,2583


In [255]:
md52rawtopic = {}
for rowid, row in tqdm(df_raw.iterrows(), total=len(df_raw)):
    md52rawtopic[row['Md5']] = row['Topic']

100%|████████████████████████████████████████████| 531291/531291 [00:18<00:00, 29377.44it/s]


In [256]:
json.dump(md52rawtopic, open('./md52rawtopic.json', 'w'), indent=4)

In [4]:
df_raw = df_raw.rename(columns={'Subject': 'Actor1Name', 'Object': 'Actor2Name', 'Relation_id': 'EventCode', 'Date': 'day', 'Nday': 'timid'})

In [5]:
df_raw

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,Topic,timid
0,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0
1,US,057,Sign formal agreement,Iran,0b6cd037f3fc22ed82e0c147839ba9f9,20150219,15,0
2,United States,051,Praise or endorse,Israel,0e3390e4d38e679fc25d764528a781e4,20150219,-1,0
3,United States,111,Criticize or denounce,Islam,0b79f78beb4ffba834ab552c7ef020ea,20150219,-1,0
4,Hamas,1831,Carry out suicide bombing,Israel,859fc96157484e73268e105e6222ce53,20150219,852,0
...,...,...,...,...,...,...,...,...
531286,Russia,061,Cooperate economically,Ukraine,3f48fb321a6a8e00f5f8a2c1fd756b87,20220317,1203,2583
531287,Ukraine,080,Yield or concede,Israel,33469c03a101913442242e89f18899ce,20220317,28,2583
531288,Israel,061,Cooperate economically,Ukraine,9e24beca44f3a9d8a966625f97b55814,20220317,28,2583
531289,Ukraine,194,Fight with artillery and tanks,Russia,cac2d7644682d7109ca0d2d7a5aac8d5,20220317,24,2583


In [47]:
grouplist = list(df_raw.groupby(['Actor1Name', 'EventCode', 'Actor2Name', 'day', 'Topic', 'timid']))

In [49]:
grouplist[4700][1]

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,Topic,timid
115965,Ali Akbar Velayati,0311,Express intent to cooperate economically,Michel Aoun,df98fbc467047f3676979164307e93f0,20161112,763,632


In [50]:
ce_pds = []
for group in grouplist:
    group_df = group[1]
    md5_list = list(group_df['Md5'].unique())
    ce_pd = group_df.head(1).copy()
    ce_pd['Md5_list'] = ', '.join(md5_list)
    ce_pds.append(ce_pd.copy())

In [48]:
md5_list

['1fea953741f1263f9db0b936cbb21c39']

In [158]:
ce_df = pd.concat(ce_pds).sort_index()

In [159]:
ce_df

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,Topic,timid,Md5_list
0,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0,029d9dc7a91b489457e962f4d6bc0fe3
1,US,057,Sign formal agreement,Iran,0b6cd037f3fc22ed82e0c147839ba9f9,20150219,15,0,0b6cd037f3fc22ed82e0c147839ba9f9
2,United States,051,Praise or endorse,Israel,0e3390e4d38e679fc25d764528a781e4,20150219,-1,0,0e3390e4d38e679fc25d764528a781e4
3,United States,111,Criticize or denounce,Islam,0b79f78beb4ffba834ab552c7ef020ea,20150219,-1,0,0b79f78beb4ffba834ab552c7ef020ea
4,Hamas,1831,Carry out suicide bombing,Israel,859fc96157484e73268e105e6222ce53,20150219,852,0,859fc96157484e73268e105e6222ce53
...,...,...,...,...,...,...,...,...,...
531285,Italy,013,Make optimistic comment,Ukraine,ea22d1bdc80260654b06df8de7f20b9d,20220317,-1,2583,ea22d1bdc80260654b06df8de7f20b9d
531286,Russia,061,Cooperate economically,Ukraine,3f48fb321a6a8e00f5f8a2c1fd756b87,20220317,1203,2583,3f48fb321a6a8e00f5f8a2c1fd756b87
531287,Ukraine,080,Yield or concede,Israel,33469c03a101913442242e89f18899ce,20220317,28,2583,33469c03a101913442242e89f18899ce
531289,Ukraine,194,Fight with artillery and tanks,Russia,cac2d7644682d7109ca0d2d7a5aac8d5,20220317,24,2583,cac2d7644682d7109ca0d2d7a5aac8d5


In [160]:
len(ce_df['Md5'].unique())

254534

In [161]:
# stats
results = ce_df.groupby(['Topic']).agg({'timid':['count', 'max','min','mean']})
results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
results['nday_range'] = results['nday_max'] -results['nday_min'] + 1

In [162]:
results.loc[0:, :].mean()

n_events        56.379733
nday_max      1264.983467
nday_min      1226.736800
nday_mean     1245.206599
nday_range      39.246667
dtype: float64

In [163]:
results.loc[0:, :].max()

n_events      3879.000000
nday_max      2583.000000
nday_min      2574.000000
nday_mean     2578.434783
nday_range    2568.000000
dtype: float64

In [164]:
results.loc[0:, :].min()

n_events       6.000000
nday_max      18.000000
nday_min       0.000000
nday_mean      6.070175
nday_range     2.000000
dtype: float64

In [58]:
max_n_range = 78
max_n_events = 112

In [165]:
topicid_max = ce_df['Topic'].max()

In [166]:
topicid_max

3749

In [167]:
ce_pd_splitted = []
for topicid in range(0, topicid_max + 1):
    ce_pd = ce_df[ce_df['Topic']==topicid]
    ce_pd = ce_pd.sort_values(by=['day'], ignore_index=True)
    curr_rowid = 0
    while curr_rowid <= (len(ce_pd)-1):
        next_rowid = min(curr_rowid + max_n_events-1, len(ce_pd)-1)
        next_row = ce_pd.iloc[[next_rowid]]
        next_timid = next_row['timid'].values[0]
        curr_row = ce_pd.iloc[[curr_rowid]]
        curr_timid = curr_row['timid'].values[0]
        n_range = next_timid - curr_timid + 1
        if n_range <= max_n_range: # split by max_n_events
            ce_pd_splitted.append(ce_pd[curr_rowid: next_rowid+1])
            curr_rowid = next_rowid+1
        else: # split by n_range
            next_rowid = len(ce_pd[ce_pd['timid'] <= (curr_timid+max_n_range-1)]) - 1
            ce_pd_splitted.append(ce_pd[curr_rowid: next_rowid+1])
            curr_rowid = next_rowid+1    

In [168]:
new_ces = []
for idx, new_ce in enumerate(ce_pd_splitted):
    new_ce = new_ce.rename(columns={'Topic': 'ce_id'})
    new_ce['ce_id'] = [idx] * len(new_ce)
    new_ces.append(new_ce)

In [169]:
len(new_ces)

4662

In [170]:
new_ces[0]

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,Egypt,012,Make pessimistic comment,China,3d52c8acaab22b919f627ad1fc0d3b70,20200123,0,1799,3d52c8acaab22b919f627ad1fc0d3b70
1,Iran,081,Ease administrative sanctions,Covid-19,a39827d96353800f37ffa5dbc76f0461,20200126,0,1802,a39827d96353800f37ffa5dbc76f0461
2,Israel,081,Ease administrative sanctions,China,ec804b29897268702f74ea6b1de45482,20200126,0,1802,ec804b29897268702f74ea6b1de45482
3,Ben Gurion Airport,054,Grant diplomatic recognition,China,ec804b29897268702f74ea6b1de45482,20200126,0,1802,ec804b29897268702f74ea6b1de45482
4,El Al airlines,013,Make optimistic comment,China,23688a6abaa4031cf0fbd4d6d1a7f370,20200126,0,1802,23688a6abaa4031cf0fbd4d6d1a7f370
...,...,...,...,...,...,...,...,...,...
107,Egypt,011,Refuse to comment,Egypt government,90e0cbc9ac6076127a2ef756ac219c6f,20200214,0,1821,90e0cbc9ac6076127a2ef756ac219c6f
108,Egypt,013,Make optimistic comment,World Health Organization (WHO),8ed93de4c0193a582db989d11e292d82,20200214,0,1821,"8ed93de4c0193a582db989d11e292d82, dbda75c64989..."
109,China,080,Yield or concede,France,cd782f378305bb4ba0a88725b9dc9658,20200215,0,1822,cd782f378305bb4ba0a88725b9dc9658
110,China,080,Yield or concede,Egypt,cd782f378305bb4ba0a88725b9dc9658,20200215,0,1822,cd782f378305bb4ba0a88725b9dc9658


In [171]:
outliers = ce_df[ce_df['Topic']==-1].sort_values(by=['day'], ignore_index=True)
outliers = outliers.rename(columns={'Topic': 'ce_id'})

In [172]:
outliers

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0,029d9dc7a91b489457e962f4d6bc0fe3
1,Egypt,062,Cooperate militarily,Israel,61f06c16f8cca2adb60f2f62c1da52b0,20150219,-1,0,61f06c16f8cca2adb60f2f62c1da52b0
2,Hezbollah,1431,Conduct strike or boycott for leadership change,Qatar,2604cc86179d28276f48d69594531be3,20150219,-1,0,2604cc86179d28276f48d69594531be3
3,Egypt,061,Cooperate economically,Jordan,fea4f048e58509eb5ce3a95a356d3f3f,20150219,-1,0,fea4f048e58509eb5ce3a95a356d3f3f
4,Egypt,1011,Demand economic cooperation,Qatar,16870399ca4d9cf49252b06e533928ef,20150219,-1,0,16870399ca4d9cf49252b06e533928ef
...,...,...,...,...,...,...,...,...,...
246187,Israel,060,Engage in material cooperation,UAE,ba8a8abcabd3cb7443edefae532487f6,20220317,-1,2583,ba8a8abcabd3cb7443edefae532487f6
246188,Palestinian Authority,0313,Express intent to cooperate on judicial matters,International Criminal Court (ICC),d52355726f828f51f7deaa065f4fa451,20220317,-1,2583,d52355726f828f51f7deaa065f4fa451
246189,US,111,Criticize or denounce,Iran,5ef9ffb39293cb308285c6259b124392,20220317,-1,2583,5ef9ffb39293cb308285c6259b124392
246190,Ukraine,085,"Ease economic sanction, boycott, or embargo",Russia,c883a848a44a76fd9356c60cae74522d,20220317,-1,2583,c883a848a44a76fd9356c60cae74522d


In [173]:
new_ces.append(outliers)
new_ces_df = pd.concat(new_ces, ignore_index=True)

In [174]:
new_ces_df = new_ces_df.sort_values(by=['day'], ignore_index=True)

In [175]:
new_ces_df

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,Iran,031,Express intent to engage in material cooperation,Russia,3a4bb7378ab66388eee2ab5657ae794f,20150219,-1,0,3a4bb7378ab66388eee2ab5657ae794f
1,United States,011,Refuse to comment,Iran government,81089d58c9402bcf29f1ffeb2a57e525,20150219,-1,0,81089d58c9402bcf29f1ffeb2a57e525
2,Israel,193,Fight with small arms and light weapons,Lebanon,93de594238f1e46906cb73bfc368a219,20150219,-1,0,93de594238f1e46906cb73bfc368a219
3,Israel,013,Make optimistic comment,Jordan,b8c6ea9a1d38101ce4d5f8ea673b404a,20150219,-1,0,b8c6ea9a1d38101ce4d5f8ea673b404a
4,King Hussein,013,Make optimistic comment,Israel,b8c6ea9a1d38101ce4d5f8ea673b404a,20150219,-1,0,b8c6ea9a1d38101ce4d5f8ea673b404a
...,...,...,...,...,...,...,...,...,...
457611,Russia,050,Engage in diplomatic cooperation,Ukraine,cc0794c46588af7a547caea72ba95c27,20220317,193,2583,cc0794c46588af7a547caea72ba95c27
457612,Israel,060,Engage in material cooperation,Ukraine,daa3e25a5671fd7683ed957f7e3c8972,20220317,193,2583,daa3e25a5671fd7683ed957f7e3c8972
457613,Ukraine,030,Express intent to cooperate,Israel,87a046d58f0a05e2c9c1bb9bd826b594,20220317,193,2583,87a046d58f0a05e2c9c1bb9bd826b594
457614,Ukraine,080,Yield or concede,Russia,c43e61753acf9cd4c4617109f7130ae5,20220317,193,2583,c43e61753acf9cd4c4617109f7130ae5


In [176]:
new_ces_df['ce_id'].max()

4661

In [177]:
# stats
new_results = new_ces_df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
new_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
new_results['nday_range'] = new_results['nday_max'] -new_results['nday_min'] + 1

In [178]:
new_results

,n_events,nday_max,nday_min,nday_mean,nday_range
ce_id,,,,,
-1,246192,2583,0,1217.097461,2584
0,112,1822,1799,1809.776786,24
1,112,1829,1823,1826.303571,7
2,112,1831,1829,1829.830357,3
3,112,1832,1831,1831.223214,2
...,...,...,...,...,...
4657,18,2474,2444,2459.222222,31
4658,14,1392,1359,1373.928571,34
4659,13,1319,1286,1304.230769,34


In [179]:
new_results.loc[0:, :].mean()

n_events        45.350493
nday_max      1290.744959
nday_min      1261.438438
nday_mean     1275.922256
nday_range      30.306521
dtype: float64

In [180]:
new_results.loc[0:, :].max()

n_events       112.000000
nday_max      2583.000000
nday_min      2582.000000
nday_mean     2582.705882
nday_range      78.000000
dtype: float64

In [181]:
new_results.loc[0:, :].min()

n_events      1.000000
nday_max      9.000000
nday_min      0.000000
nday_mean     4.642857
nday_range    1.000000
dtype: float64

In [ ]:
# filter by min range and min size

In [182]:
unfiltered_ces = new_ces[:-1]

In [183]:
invalid_ces = []
valid_ces = []
for ce in unfiltered_ces:
    if len(ce) < 10:
        invalid_ces.append(ce)
    elif len(ce['timid'].unique()) == 1:
        invalid_ces.append(ce)
    else:
        valid_ces.append(ce)

In [184]:
len(invalid_ces)

261

In [185]:
new_outliers = pd.concat([outliers]+invalid_ces, ignore_index=True)
new_outliers['ce_id'] = [-1] * len(new_outliers)

In [186]:
new_outliers

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0,029d9dc7a91b489457e962f4d6bc0fe3
1,Egypt,062,Cooperate militarily,Israel,61f06c16f8cca2adb60f2f62c1da52b0,20150219,-1,0,61f06c16f8cca2adb60f2f62c1da52b0
2,Hezbollah,1431,Conduct strike or boycott for leadership change,Qatar,2604cc86179d28276f48d69594531be3,20150219,-1,0,2604cc86179d28276f48d69594531be3
3,Egypt,061,Cooperate economically,Jordan,fea4f048e58509eb5ce3a95a356d3f3f,20150219,-1,0,fea4f048e58509eb5ce3a95a356d3f3f
4,Egypt,1011,Demand economic cooperation,Qatar,16870399ca4d9cf49252b06e533928ef,20150219,-1,0,16870399ca4d9cf49252b06e533928ef
...,...,...,...,...,...,...,...,...,...
247661,China,061,Cooperate economically,Egypt,c13e8717c56950a27b5ca2495fe38022,20190630,-1,1592,c13e8717c56950a27b5ca2495fe38022
247662,Egypt,0211,Appeal for economic cooperation,China,3711ab09654a830db32b0a9afee8cbe4,20190701,-1,1593,3711ab09654a830db32b0a9afee8cbe4
247663,China,061,Cooperate economically,Egypt,71d3f11c0eef3fb084ef8657cd6fda61,20190704,-1,1596,"71d3f11c0eef3fb084ef8657cd6fda61, c2eb9c45c03a..."
247664,China,013,Make optimistic comment,Egypt,905421f98fd7cf0573d5fb128525a87f,20190709,-1,1601,905421f98fd7cf0573d5fb128525a87f


In [187]:
new_outliers_grouplist = list(new_outliers.groupby(['Actor1Name', 'EventCode', 'Actor2Name', 'day']))
print(len(new_outliers_grouplist))
new_outliers_dedu_list = []
for groupid, group in tqdm(enumerate(new_outliers_grouplist), total=len(new_outliers_grouplist)):
    group_df = group[1]
    md5_list = []
    md5_lists = list(group_df['Md5_list'].unique())
    for md5_list_str in md5_lists:
        md5_list += md5_list_str.split(', ')
    kept_outlier = group_df.head(1).copy()
    kept_outlier['Md5_list'] = ', '.join(md5_list)
    new_outliers_dedu_list.append(kept_outlier.copy())
new_outliers_dedu = pd.concat(new_outliers_dedu_list).sort_values(by=['day'], ignore_index=True)

247560


100%|█████████████████████████████████████████████| 247560/247560 [01:14<00:00, 3300.92it/s]


In [188]:
new_outliers_dedu

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,Britain,011,Refuse to comment,United Nations,1b20f22d197f24368a2dfda404401d59,20150219,-1,0,1b20f22d197f24368a2dfda404401d59
1,Israel,050,Engage in diplomatic cooperation,Egypt,b64c9338979108ce15043ad6f21a8a17,20150219,-1,0,b64c9338979108ce15043ad6f21a8a17
2,EU,070,Provide aid,US,72dcb54c9b49830caefdad4bb7453d3c,20150219,-1,0,72dcb54c9b49830caefdad4bb7453d3c
3,Egypt,180,Use unconventional violence including terrorist,Israel,61f06c16f8cca2adb60f2f62c1da52b0,20150219,-1,0,61f06c16f8cca2adb60f2f62c1da52b0
4,Arab League Council,016,"Reject accusation, deny responsibility",Yemen,d19f42d1f5e86ea8101e933b2668ed99,20150219,-1,0,d19f42d1f5e86ea8101e933b2668ed99
...,...,...,...,...,...,...,...,...,...
247555,Robert Malley,092,Investigate human rights abuses,NSO Group,2ae0e2446612a3fe99ec1ee481e01ffa,20220317,-1,2583,2ae0e2446612a3fe99ec1ee481e01ffa
247556,Emmanuel Macron,202,Engage in mass killings,NSO Group,2ae0e2446612a3fe99ec1ee481e01ffa,20220317,-1,2583,2ae0e2446612a3fe99ec1ee481e01ffa
247557,Ukraine,121,Reject material cooperation,Russia,c883a848a44a76fd9356c60cae74522d,20220317,-1,2583,c883a848a44a76fd9356c60cae74522d
247558,Russia,060,Engage in material cooperation,Ukraine,4736fdcba416ec16e321dcded9d5f41e,20220317,-1,2583,4736fdcba416ec16e321dcded9d5f41e


In [189]:
new_outliers_dedu[new_outliers_dedu['timid']==0]

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,Britain,011,Refuse to comment,United Nations,1b20f22d197f24368a2dfda404401d59,20150219,-1,0,1b20f22d197f24368a2dfda404401d59
1,Israel,050,Engage in diplomatic cooperation,Egypt,b64c9338979108ce15043ad6f21a8a17,20150219,-1,0,b64c9338979108ce15043ad6f21a8a17
2,EU,070,Provide aid,US,72dcb54c9b49830caefdad4bb7453d3c,20150219,-1,0,72dcb54c9b49830caefdad4bb7453d3c
3,Egypt,180,Use unconventional violence including terrorist,Israel,61f06c16f8cca2adb60f2f62c1da52b0,20150219,-1,0,61f06c16f8cca2adb60f2f62c1da52b0
4,Arab League Council,016,"Reject accusation, deny responsibility",Yemen,d19f42d1f5e86ea8101e933b2668ed99,20150219,-1,0,d19f42d1f5e86ea8101e933b2668ed99
...,...,...,...,...,...,...,...,...,...
129,India,0312,Express intent to cooperate militarily,Firms and Markets,1db3eecab31a2ab29713c6d042631ace,20150219,-1,0,1db3eecab31a2ab29713c6d042631ace
130,European Union,071,Provide economic aid,Palestinian Authority,029d9dc7a91b489457e962f4d6bc0fe3,20150219,-1,0,029d9dc7a91b489457e962f4d6bc0fe3
131,Palestine,0831,Accede to demands for change in leadership,Italy,e6d0a477fced53099b4e48519922ccfb,20150219,-1,0,e6d0a477fced53099b4e48519922ccfb
132,Israel,160,Reduce relations,Egypt,b64c9338979108ce15043ad6f21a8a17,20150219,-1,0,b64c9338979108ce15043ad6f21a8a17


In [190]:
len(new_outliers_dedu['Md5'].unique())

137964

In [191]:
for idx, valid_ce in enumerate(valid_ces):
    valid_ce['ce_id'] = [idx] * len(valid_ce)

In [192]:
filtered_ces = valid_ces + [new_outliers_dedu]
filtered_ces_df = pd.concat(filtered_ces, ignore_index=True)

In [193]:
filtered_ces_df = filtered_ces_df.sort_values(by=['day'], ignore_index=True)

In [194]:
filtered_ces_df

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,Palestine,0831,Accede to demands for change in leadership,Italy,e6d0a477fced53099b4e48519922ccfb,20150219,-1,0,e6d0a477fced53099b4e48519922ccfb
1,Israel,060,Engage in material cooperation,Egypt,b64c9338979108ce15043ad6f21a8a17,20150219,-1,0,b64c9338979108ce15043ad6f21a8a17
2,Israel,180,Use unconventional violence including terrorist,Egypt,b64c9338979108ce15043ad6f21a8a17,20150219,-1,0,b64c9338979108ce15043ad6f21a8a17
3,United States,111,Criticize or denounce,Israel,d3f9a383ef3913ddb61b655b43841ae7,20150219,-1,0,d3f9a383ef3913ddb61b655b43841ae7
4,United States,111,Criticize or denounce,Islam,0b79f78beb4ffba834ab552c7ef020ea,20150219,-1,0,0b79f78beb4ffba834ab552c7ef020ea
...,...,...,...,...,...,...,...,...,...
457505,Ukraine,140,Engage in political dissent,Russia,dafe15f7bc95e33b171b5663fc55b735,20220317,221,2583,dafe15f7bc95e33b171b5663fc55b735
457506,Lithuania,021,Appeal for material cooperation,Ukraine,dafe15f7bc95e33b171b5663fc55b735,20220317,221,2583,dafe15f7bc95e33b171b5663fc55b735
457507,United States,101,Demand material cooperation,Iran,2886e5946912e59c391e41a238276bb7,20220317,2715,2583,2886e5946912e59c391e41a238276bb7
457508,Lithuania,032,Express intent to engage in diplomatic coopera...,Ukraine,dafe15f7bc95e33b171b5663fc55b735,20220317,221,2583,dafe15f7bc95e33b171b5663fc55b735


In [195]:
len(filtered_ces_df['Md5'].unique())

254456

In [196]:
len(filtered_ces_df[filtered_ces_df['ce_id']!=-1]['Md5'].unique())

116550

In [197]:
# stats
filtered_results = filtered_ces_df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
filtered_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
filtered_results['nday_range'] = filtered_results['nday_max'] -filtered_results['nday_min'] + 1

In [198]:
filtered_results

,n_events,nday_max,nday_min,nday_mean,nday_range
ce_id,,,,,
-1,247560,2583,0,1217.775416,2584
0,112,1822,1799,1809.776786,24
1,112,1829,1823,1826.303571,7
2,112,1831,1829,1829.830357,3
3,112,1832,1831,1831.223214,2
...,...,...,...,...,...
4396,18,2474,2444,2459.222222,31
4397,14,1392,1359,1373.928571,34
4398,13,1319,1286,1304.230769,34


In [199]:
filtered_results.loc[0:, :].mean()

n_events        47.705067
nday_max      1284.648489
nday_min      1254.556692
nday_mean     1269.423288
nday_range      31.091797
dtype: float64

In [200]:
filtered_results.loc[0:, :].max()

n_events       112.000000
nday_max      2583.000000
nday_min      2582.000000
nday_mean     2582.705882
nday_range      78.000000
dtype: float64

In [201]:
filtered_results.loc[0:, :].min()

n_events      10.000000
nday_max       9.000000
nday_min       0.000000
nday_mean      4.642857
nday_range     2.000000
dtype: float64

In [202]:
filtered_ces_df.to_csv(path_or_buf='./EGIRIS_ce_raw.csv', sep='\t', index=False)

In [ ]:
# create needed files

In [203]:
unique_time = filtered_ces_df['day'].unique()
tim2id = {name: idx for idx, name in enumerate(unique_time)}

In [204]:
with open('./date2id.txt', 'w') as f:
    for k, v in tim2id.items():
        f.write(f"{k}\t{v}\n")

In [205]:
tim2id

{20150219: 0,
 20150220: 1,
 20150221: 2,
 20150222: 3,
 20150223: 4,
 20150224: 5,
 20150225: 6,
 20150226: 7,
 20150227: 8,
 20150228: 9,
 20150301: 10,
 20150302: 11,
 20150303: 12,
 20150304: 13,
 20150305: 14,
 20150306: 15,
 20150307: 16,
 20150308: 17,
 20150309: 18,
 20150310: 19,
 20150311: 20,
 20150312: 21,
 20150313: 22,
 20150314: 23,
 20150315: 24,
 20150316: 25,
 20150317: 26,
 20150318: 27,
 20150319: 28,
 20150320: 29,
 20150321: 30,
 20150322: 31,
 20150323: 32,
 20150324: 33,
 20150325: 34,
 20150326: 35,
 20150327: 36,
 20150328: 37,
 20150329: 38,
 20150330: 39,
 20150331: 40,
 20150401: 41,
 20150402: 42,
 20150403: 43,
 20150404: 44,
 20150405: 45,
 20150406: 46,
 20150407: 47,
 20150408: 48,
 20150409: 49,
 20150410: 50,
 20150411: 51,
 20150412: 52,
 20150413: 53,
 20150414: 54,
 20150415: 55,
 20150416: 56,
 20150417: 57,
 20150418: 58,
 20150419: 59,
 20150420: 60,
 20150421: 61,
 20150422: 62,
 20150423: 63,
 20150424: 64,
 20150425: 65,
 20150426: 66,
 2015

In [ ]:
# filtered_ces = valid_ces + [new_outliers_dedu]

In [206]:
test_max_date = filtered_ces_df['day'].max()
test_min_date = test_max_date-10000+1 # one year
val_max_date = test_min_date - 1
val_min_date = val_max_date-10000+1 # one year
train_max_date = val_min_date-1

In [207]:
train_max_date

20200317

In [ ]:
# train_outliters = new_outliers_dedu[new_outliers_dedu['day'] <= train_max_date]
# val_outliters = new_outliers_dedu[(new_outliers_dedu['day'] >= val_min_date) & (new_outliers_dedu['day'] <= val_max_date)]
# test_outliters = new_outliers_dedu[new_outliers_dedu['day'] >= test_min_date]
# for _ in [train_outliters, val_outliters, test_outliters]:
#     print(len(_))
#     print(len(_['Md5'].unique()))

In [208]:
# split ces
ceid2centertimid = {}
for ceid, valid_ce in enumerate(valid_ces):
    unique_timids = list(valid_ce['timid'].unique())
    timid_sum = 0
    for timid in unique_timids:
        timid_sum += timid * len(valid_ce[valid_ce['timid']==timid])
    centertimid = timid_sum*1.0/len(valid_ce)
    ceid2centertimid[ceid] = centertimid

In [209]:
ceid2centertimid

{0: 1809.7767857142858,
 1: 1826.3035714285713,
 2: 1829.830357142857,
 3: 1831.2232142857142,
 4: 1832.7589285714287,
 5: 1834.6875,
 6: 1836.705357142857,
 7: 1838.4464285714287,
 8: 1840.357142857143,
 9: 1841.9375,
 10: 1843.767857142857,
 11: 1845.7857142857142,
 12: 1847.5625,
 13: 1848.6785714285713,
 14: 1851.1875,
 15: 1853.4017857142858,
 16: 1854.5892857142858,
 17: 1856.794642857143,
 18: 1859.232142857143,
 19: 1863.4017857142858,
 20: 1867.25,
 21: 1871.0267857142858,
 22: 1874.767857142857,
 23: 1880.044642857143,
 24: 1887.2767857142858,
 25: 1893.9285714285713,
 26: 1899.6339285714287,
 27: 1910.7767857142858,
 28: 1922.625,
 29: 1934.142857142857,
 30: 1948.1607142857142,
 31: 1960.8214285714287,
 32: 1970.5982142857142,
 33: 1981.375,
 34: 1996.394366197183,
 35: 231.0,
 36: 242.33035714285714,
 37: 254.52678571428572,
 38: 271.1875,
 39: 286.4375,
 40: 304.0892857142857,
 41: 318.04464285714283,
 42: 324.55357142857144,
 43: 332.33035714285717,
 44: 341.75,
 45: 357

In [210]:
train_ceids, val_ceids, test_ceids = [], [], []
for ceid, centertimid in ceid2centertimid.items():
    if centertimid <= tim2id[train_max_date]:
        train_ceids.append(ceid)
    elif centertimid >= tim2id[val_min_date] and centertimid <= tim2id[val_max_date]:
        val_ceids.append(ceid)
    else:
        test_ceids.append(ceid)

In [211]:
train_ce_df = pd.concat([valid_ces[ceid] for ceid in train_ceids], ignore_index=True)
val_ce_df = pd.concat([valid_ces[ceid] for ceid in val_ceids], ignore_index=True)
test_ce_df = pd.concat([valid_ces[ceid] for ceid in test_ceids], ignore_index=True)

In [ ]:
# this is wrong because ceid in valid_ces is not continuous
# train_ce_df = filtered_ces_df[filtered_ces_df['ce_id'].isin(train_ceids)]
# val_ce_df = filtered_ces_df[filtered_ces_df['ce_id'].isin(val_ceids)]
# test_ce_df = filtered_ces_df[filtered_ces_df['ce_id'].isin(val_ceids)]

In [212]:
print(len(train_ceids))
print(len(val_ceids))
print(len(test_ceids))

3452
494
455


In [213]:
train_df = train_ce_df.sort_values(by=['day'], ignore_index=True)
val_df = val_ce_df.sort_values(by=['day'], ignore_index=True)
test_df = test_ce_df.sort_values(by=['day'], ignore_index=True)

In [214]:
# assign new ceids
old_ceids = list(train_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid
train_df['ce_id'] = [mapping[_] for _ in train_df['ce_id']]

In [215]:
old_ceids = list(val_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(train_df['ce_id']) + 1
val_df['ce_id'] = [mapping[_] for _ in val_df['ce_id']]

In [216]:
old_ceids = list(test_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(val_df['ce_id']) + 1
test_df['ce_id'] = [mapping[_] for _ in test_df['ce_id']]

In [217]:
all_df = pd.concat([train_df, val_df, test_df, new_outliers_dedu]).sort_values(by=['day'], ignore_index=True)

In [218]:
# stats
for df in [all_df]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
254456
Total events
457510
Number of entities
2849
Number of relations
238
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       247560      2583         0  1217.775416        2584
 0          112        12         0     4.642857          13
 1           53        29         0    13.094340          30
 2           57        76         0    22.807018          77
 3           57        28         0     6.070175          29
...         ...       ...       ...          ...         ...
 4396        46      2583      2577  2579.195652           7
 4397       112      2582      2578  2579.866071           5
 4398        53      2583      2582  2582.679245           2
 4399        53      2583      2582  2582.471698           2
 4400        34      2583      2582  2582.705882           2

[4402 rows x 5 columns]
n_events        47.705067
nday_max      1284.648489
nday_min      1254.556692
nday_mean     1269

In [219]:
# stats
for df in [train_df, val_df, test_df, new_outliers_dedu]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
90252
Total events
160953
Number of entities
2641
Number of relations
218
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
0           112        12         0     4.642857          13
1            53        29         0    13.094340          30
2            57        76         0    22.807018          77
3            57        28         0     6.070175          29
4           112        11         0     6.562500          12
...         ...       ...       ...          ...         ...
3447         24      1853      1846  1849.750000           8
3448        112      1848      1847  1847.562500           2
3449         28      1854      1847  1849.321429           8
3450        112      1850      1848  1848.678571           3
3451        112      1852      1850  1851.187500           3

[3452 rows x 5 columns]
n_events        46.626014
nday_max      1024.823001
nday_min       995.708285
nday_mean     1010.

In [220]:
for df in [train_df, val_df, test_df]:
    print(len(df))
    print(len(df['Md5'].unique()))
    print(df['day'].min())
    print(df['day'].max())
    print()

160953
90252
20150219
20200418

25895
13733
20200218
20210413

23102
12590
20200316
20220317



In [221]:
train_df.to_csv(path_or_buf='./EGIRIS_ce_train.csv', sep='\t', index=False)
val_df.to_csv(path_or_buf='./EGIRIS_ce_val.csv', sep='\t', index=False)
test_df.to_csv(path_or_buf='./EGIRIS_ce_test.csv', sep='\t', index=False)
all_df.to_csv(path_or_buf='./EGIRIS_ce.csv', sep='\t', index=False)

In [222]:
new_outliers_dedu = new_outliers_dedu.sort_values(by=['day'], ignore_index=True)
new_outliers_dedu.to_csv(path_or_buf='./EGIRIS_ce_outliers.csv', sep='\t', index=False)

In [ ]:
# filter unseen entity and relation after 20200317

In [223]:
anchor_df = all_df[all_df['day']<=20200317]
anchor_ent = (anchor_df['Actor1Name']._append(anchor_df['Actor2Name'])).unique()
anchor_rel = anchor_df['EventCode'].unique()

In [224]:
anchor_ent

array(['US', 'Israel', 'Italy', ..., 'COVID-19 patients',
       'Health officials', 'COVID-19 vaccine'], dtype=object)

In [226]:
len(anchor_ent)

2794

In [225]:
anchor_rel

array(['057', '1384', '140', '060', '013', '1224', '1431', '074', '0311',
       '011', '021', '034', '090', '061', '194', '183', '1222', '111',
       '101', '072', '100', '160', '0831', '071', '0312', '051', '0313',
       '050', '170', '046', '112', '0811', '073', '010', '031', '082',
       '200', '054', '180', '040', '201', '1011', '032', '016', '070',
       '190', '202', '030', '062', '0212', '022', '193', '012', '020',
       '093', '1831', '195', '041', '0332', '044', '154', '080', '1223',
       '014', '0314', '092', '110', '063', '091', '1412', '1052', '019',
       '1421', '052', '1383', '122', '191', '1711', '175', '0331', '1221',
       '042', '134', '1321', '132', '0213', '0211', '1411', '182', '121',
       '1834', '039', '081', '017', '1012', '1123', '033', '1832', '102',
       '1211', '0241', '1432', '2041', '018', '1622', '015', '1051',
       '0231', '0832', '0841', '0242', '164', '0333', '1422', '130',
       '1234', '162', '171', '104', '1233', '123', '0351', '14

In [227]:
len(anchor_rel)

234

In [228]:
seen_val_df = val_df[val_df['EventCode'].isin(anchor_rel)]
seen_test_df = test_df[test_df['EventCode'].isin(anchor_rel)]
seen_val_df = seen_val_df[(seen_val_df['Actor1Name'].isin(anchor_ent)) & (seen_val_df['Actor2Name'].isin(anchor_ent))]
seen_test_df = seen_test_df[(test_df['Actor1Name'].isin(anchor_ent)) & (seen_test_df['Actor2Name'].isin(anchor_ent))]

seen_outliers = all_df[all_df['ce_id']==-1]
seen_outliers = seen_outliers[seen_outliers['EventCode'].isin(anchor_rel)]
seen_outliers = seen_outliers[(seen_outliers['Actor1Name'].isin(anchor_ent)) & (seen_outliers['Actor2Name'].isin(anchor_ent))]

/tmp/ipykernel_2945794/17323742.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  seen_test_df = seen_test_df[(test_df['Actor1Name'].isin(anchor_ent)) & (seen_test_df['Actor2Name'].isin(anchor_ent))]


In [229]:
seen_df = pd.concat([train_df, seen_val_df, seen_test_df, seen_outliers]).sort_values(by=['day'], ignore_index=True)

In [230]:
# stats
for df in [seen_df]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
253836
Total events
455877
Number of entities
2794
Number of relations
234
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       247063      2583         0  1215.643063        2584
 0          112        12         0     4.642857          13
 1           53        29         0    13.094340          30
 2           57        76         0    22.807018          77
 3           57        28         0     6.070175          29
...         ...       ...       ...          ...         ...
 4396        44      2583      2577  2579.250000           7
 4397       112      2582      2578  2579.866071           5
 4398        52      2583      2582  2582.692308           2
 4399        52      2583      2582  2582.480769           2
 4400        34      2583      2582  2582.705882           2

[4402 rows x 5 columns]
n_events        47.446944
nday_max      1284.637355
nday_min      1254.563736
nday_mean     1269

In [231]:
# stats
for df in [seen_val_df, seen_test_df, seen_outliers]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
13542
Total events
25415
Number of entities
1284
Number of relations
176
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
3452         67      1890      1825  1860.791045          66
3453         80      1904      1829  1878.475000          76
3454         19      1907      1832  1874.631579          76
3455         19      1910      1833  1889.368421          78
3456         47      1884      1834  1867.148936          51
...         ...       ...       ...          ...         ...
3941         22      2235      2203  2210.090909          33
3942         29      2233      2203  2215.379310          31
3943         40      2223      2204  2212.100000          20
3944         22      2234      2206  2215.727273          29
3945         33      2219      2212  2213.939394           8

[494 rows x 5 columns]
n_events        51.447368
nday_max      2055.419028
nday_min      2022.093117
nday_mean     2038.61

In [232]:
seen_outliers = seen_outliers.sort_values(by=['day'], ignore_index=True)

In [233]:
seen_val_df.to_csv(path_or_buf='./EGIRIS_ce_val_seen.csv', sep='\t', index=False)
seen_test_df.to_csv(path_or_buf='./EGIRIS_ce_test_seen.csv', sep='\t', index=False)
seen_outliers.to_csv(path_or_buf='./EGIRIS_ce_outliers_seen.csv', sep='\t', index=False)
seen_df.to_csv(path_or_buf='./EGIRIS_ce_seen.csv', sep='\t', index=False)

In [234]:
ceid2size = {}
ces = list(seen_val_df['ce_id'].unique())
for ce in ces:
    ceid2size[ce] = len(seen_val_df[seen_val_df['ce_id']==ce])
    if len(seen_val_df[seen_val_df['ce_id']==ce]['timid'].unique()) == 1:
        print(ce)

In [235]:
for ce, size in ceid2size.items():
    if size < 10:
        print(ce)

3626
3705


In [236]:
seen_val_df[seen_val_df['ce_id'].isin([3626, 3705])]

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
8888,Israel,011,Refuse to comment,Palestine,b560580d74d167da187e1112d90c6bf8,20200717,3626,1975,b560580d74d167da187e1112d90c6bf8
9005,Noam Chomsky,032,Express intent to engage in diplomatic coopera...,Palestine,f51a82ee25154f4a0129b48299620b5b,20200719,3626,1977,f51a82ee25154f4a0129b48299620b5b
9045,Albert Einstein,032,Express intent to engage in diplomatic coopera...,Palestine,f51a82ee25154f4a0129b48299620b5b,20200719,3626,1977,f51a82ee25154f4a0129b48299620b5b
9398,Israel,013,Make optimistic comment,World Jewry,9acd2fa9f18a2bda64e24bf1fea7161a,20200723,3626,1981,9acd2fa9f18a2bda64e24bf1fea7161a
9399,World Jewry,011,Refuse to comment,Israel,9acd2fa9f18a2bda64e24bf1fea7161a,20200723,3626,1981,9acd2fa9f18a2bda64e24bf1fea7161a
10421,IfNotNow,101,Demand material cooperation,Jewish Voice for Peace,6220152095f8721ec2af1c4bc633d45f,20200809,3626,1998,6220152095f8721ec2af1c4bc633d45f
12846,UAE,0811,Ease restrictions on political freedoms,Israel,3b50c8ea40b7444991c09141a310af47,20200829,3705,2018,3b50c8ea40b7444991c09141a310af47
13157,Israel,190,Use conventional military force,Syria,21f352836827674979a529f40c158a0a,20200901,3705,2021,21f352836827674979a529f40c158a0a
13298,UAE,060,Engage in material cooperation,Saudi Arabia,c9fc65225549d5c630f10c35d6297443,20200902,3705,2022,c9fc65225549d5c630f10c35d6297443
15011,Saudi Arabia,060,Engage in material cooperation,Ethiopia,003517d17add439ad5234e7a323ab99d,20200926,3705,2046,003517d17add439ad5234e7a323ab99d


In [237]:
ceid2size = {}
ces = list(seen_test_df['ce_id'].unique())
for ce in ces:
    ceid2size[ce] = len(seen_test_df[seen_test_df['ce_id']==ce])
    if len(seen_test_df[seen_test_df['ce_id']==ce]['timid'].unique()) == 1:
        print(ce)

4091


In [238]:
for ce, size in ceid2size.items():
    if size < 10:
        print(ce)

4091
4225


In [239]:
seen_test_df[seen_test_df['ce_id'].isin([4091, 4225])]

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
7779,Egypt,011,Refuse to comment,Japan,4bc47d7227a58990f8673cae7215048e,20210707,4091,2330,4bc47d7227a58990f8673cae7215048e
13102,NGOs,021,Appeal for material cooperation,Israel,f3a9043a322b30043ee804088f0179a3,20211013,4225,2428,f3a9043a322b30043ee804088f0179a3
13108,JTA,011,Refuse to comment,Israel,0e44946f156b9fe181b3eb0b17447e99,20211013,4225,2428,0e44946f156b9fe181b3eb0b17447e99
13137,BDS entities,011,Refuse to comment,Israel,651c966b479a9b596056dffc54b37812,20211014,4225,2429,651c966b479a9b596056dffc54b37812
13222,BDS entities,1011,Demand economic cooperation,Israelis,f475dd979e81d48acafbcfaad649e5fe,20211015,4225,2430,f475dd979e81d48acafbcfaad649e5fe
14379,Ireland,011,Refuse to comment,Israel,a980da8d3220fd58a5d1eb1c2859641c,20211105,4225,2451,a980da8d3220fd58a5d1eb1c2859641c


In [240]:
final_outliers = pd.concat([seen_outliers,
                            seen_val_df[seen_val_df['ce_id'].isin([3626, 3705])],
                            seen_test_df[seen_test_df['ce_id'].isin([4091, 4225])]])
final_outliers= final_outliers.sort_values(by=['day'], ignore_index=True)
final_outliers['ce_id'] = [-1] * len(final_outliers)

In [241]:
fianl_val_df = seen_val_df.copy()

In [242]:
fianl_val_df = fianl_val_df[~fianl_val_df['ce_id'].isin([3626, 3705])]

In [243]:
old_ceids = list(fianl_val_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(train_df['ce_id']) + 1
fianl_val_df['ce_id'] = [mapping[_] for _ in fianl_val_df['ce_id']]

In [244]:
final_test_df = seen_test_df.copy()
final_test_df = final_test_df[~final_test_df['ce_id'].isin([4091, 4225])]

In [245]:
old_ceids = list(final_test_df['ce_id'].unique())
mapping = {}
for new_ceid, ceid in enumerate(old_ceids):
    mapping[ceid] = new_ceid + max(fianl_val_df['ce_id']) + 1
final_test_df['ce_id'] = [mapping[_] for _ in final_test_df['ce_id']]

In [246]:
final_val_df = fianl_val_df.copy()

In [247]:
final_df = pd.concat([train_df, final_val_df, final_test_df, final_outliers]).sort_values(by=['day'], ignore_index=True)

In [248]:
final_df

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,US,057,Sign formal agreement,Iran,0b6cd037f3fc22ed82e0c147839ba9f9,20150219,0,0,0b6cd037f3fc22ed82e0c147839ba9f9
1,Canada government,011,Refuse to comment,Palestine,18918b73f1e5f11eb8734defabe9a804,20150219,-1,0,18918b73f1e5f11eb8734defabe9a804
2,European Union,057,Sign formal agreement,Egypt,edf3db2729f3e81b20e446b94f7a6b6a,20150219,-1,0,edf3db2729f3e81b20e446b94f7a6b6a
3,Egypt,190,Use conventional military force,Libya,1f6264640f54e7feb8e38e6968bf2dcf,20150219,-1,0,"1f6264640f54e7feb8e38e6968bf2dcf, 64af5061461b..."
4,EU,070,Provide aid,US,72dcb54c9b49830caefdad4bb7453d3c,20150219,-1,0,72dcb54c9b49830caefdad4bb7453d3c
...,...,...,...,...,...,...,...,...,...
455872,Dmitry Peskov,1222,Reject request for military aid,Ukraine,9592dd1b296aa51f5b2053e8311659e2,20220317,4395,2583,9592dd1b296aa51f5b2053e8311659e2
455873,Ukraine,093,Investigate military action,Russia,dafe15f7bc95e33b171b5663fc55b735,20220317,4395,2583,dafe15f7bc95e33b171b5663fc55b735
455874,Russia,1421,Conduct hunger strike for leadership change,Ukraine,0647d97fcb81a6f13c8b606058e4109a,20220317,4395,2583,0647d97fcb81a6f13c8b606058e4109a
455875,Ukraine,150,Exhibit military or police power,Russia,c43e61753acf9cd4c4617109f7130ae5,20220317,4396,2583,c43e61753acf9cd4c4617109f7130ae5


In [249]:
# stats
for df in [final_df]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[0:, :].mean())
    print(df_results.loc[0:, :].max())
    print(df_results.loc[0:, :].min())

Total docs
253836
Total events
455877
Number of entities
2794
Number of relations
234
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
-1       247083      2583         0  1215.717637        2584
 0          112        12         0     4.642857          13
 1           53        29         0    13.094340          30
 2           57        76         0    22.807018          77
 3           57        28         0     6.070175          29
...         ...       ...       ...          ...         ...
 4392        44      2583      2577  2579.250000           7
 4393       112      2582      2578  2579.866071           5
 4394        52      2583      2582  2582.692308           2
 4395        52      2583      2582  2582.480769           2
 4396        34      2583      2582  2582.705882           2

[4398 rows x 5 columns]
n_events        47.485558
nday_max      1283.795315
nday_min      1253.714806
nday_mean     1268

In [250]:
# stats
for df in [final_val_df, final_test_df, final_outliers]:
    print('Total docs')
    print(len(df['Md5'].unique()))
    print('Total events')
    print(len(df))
    print('Number of entities')
    print(len(df['Actor1Name']._append(df['Actor2Name']).unique()))
    print('Number of relations')
    print(len(df['EventCode'].unique()))
    
    df_results = df.groupby(['ce_id']).agg({'timid':['count', 'max','min','mean']})
    df_results.columns = ['n_events', 'nday_max', 'nday_min', 'nday_mean']
    df_results['nday_range'] = df_results['nday_max'] - df_results['nday_min'] + 1
    print(df_results)
    print(df_results.loc[:, :].mean())
    print(df_results.loc[:, :].max())
    print(df_results.loc[:, :].min())

Total docs
13532
Total events
25401
Number of entities
1281
Number of relations
176
       n_events  nday_max  nday_min    nday_mean  nday_range
ce_id                                                       
3452         67      1890      1825  1860.791045          66
3453         80      1904      1829  1878.475000          76
3454         19      1907      1832  1874.631579          76
3455         19      1910      1833  1889.368421          78
3456         47      1884      1834  1867.148936          51
...         ...       ...       ...          ...         ...
3939         22      2235      2203  2210.090909          33
3940         29      2233      2203  2215.379310          31
3941         40      2223      2204  2212.100000          20
3942         22      2234      2206  2215.727273          29
3943         33      2219      2212  2213.939394           8

[492 rows x 5 columns]
n_events        51.628049
nday_max      2055.522358
nday_min      2022.197154
nday_mean     2038.72

In [251]:
final_val_df.to_csv(path_or_buf='./EGIRIS_ce_val_final.csv', sep='\t', index=False)
final_test_df.to_csv(path_or_buf='./EGIRIS_ce_test_final.csv', sep='\t', index=False)
final_outliers.to_csv(path_or_buf='./EGIRIS_ce_outliers_final.csv', sep='\t', index=False)
final_df.to_csv(path_or_buf='./EGIRIS_ce_final.csv', sep='\t', index=False)

In [252]:
final_df

,Actor1Name,EventCode,Relation_choice,Actor2Name,Md5,day,ce_id,timid,Md5_list
0,US,057,Sign formal agreement,Iran,0b6cd037f3fc22ed82e0c147839ba9f9,20150219,0,0,0b6cd037f3fc22ed82e0c147839ba9f9
1,Canada government,011,Refuse to comment,Palestine,18918b73f1e5f11eb8734defabe9a804,20150219,-1,0,18918b73f1e5f11eb8734defabe9a804
2,European Union,057,Sign formal agreement,Egypt,edf3db2729f3e81b20e446b94f7a6b6a,20150219,-1,0,edf3db2729f3e81b20e446b94f7a6b6a
3,Egypt,190,Use conventional military force,Libya,1f6264640f54e7feb8e38e6968bf2dcf,20150219,-1,0,"1f6264640f54e7feb8e38e6968bf2dcf, 64af5061461b..."
4,EU,070,Provide aid,US,72dcb54c9b49830caefdad4bb7453d3c,20150219,-1,0,72dcb54c9b49830caefdad4bb7453d3c
...,...,...,...,...,...,...,...,...,...
455872,Dmitry Peskov,1222,Reject request for military aid,Ukraine,9592dd1b296aa51f5b2053e8311659e2,20220317,4395,2583,9592dd1b296aa51f5b2053e8311659e2
455873,Ukraine,093,Investigate military action,Russia,dafe15f7bc95e33b171b5663fc55b735,20220317,4395,2583,dafe15f7bc95e33b171b5663fc55b735
455874,Russia,1421,Conduct hunger strike for leadership change,Ukraine,0647d97fcb81a6f13c8b606058e4109a,20220317,4395,2583,0647d97fcb81a6f13c8b606058e4109a
455875,Ukraine,150,Exhibit military or police power,Russia,c43e61753acf9cd4c4617109f7130ae5,20220317,4396,2583,c43e61753acf9cd4c4617109f7130ae5


In [253]:
md5_lists = list(final_df['Md5_list'].unique())
md5s = set()
for md5_list in md5_lists:
    md5s.update(md5_list.split(', '))
print(len(md5s))

274795


In [254]:
json.dump(list(md5s), open('./final_md5_list.json', 'w'), indent=4)

In [259]:
md52ceid = {}
ceid2md5 = {}
for rowid, row in tqdm(final_df.iterrows(), total=len(final_df)):
    md5s = row['Md5_list'].split(', ')
    ceid = row['ce_id']
    if ceid not in ceid2md5:
        ceid2md5[ceid] = []
    for md5 in md5s:
        if md5 in md52ceid and md52ceid[md5]!=ceid:
            print(md5) 
        md52ceid[md5] = ceid
        ceid2md5[ceid].append(md5)

  1%|▌                                             | 5921/455877 [00:00<00:19, 22523.53it/s]

2af54da8db4d51558ef37d7c1c93f802
4b3893c314c89597698d497cc1d34611
383ac61ae8c4634f6a647706fc341750
7d3c5aa826aca441e160b7dbbfa974c1
78bc87d00a7adbe1434c95b7f90df794
44bd995be349e958f1ca5d8b0d69af18
de2ed26b88084100d72145207b3f8677
718f6d34145347bae80bc395ac3dbde7
de2ed26b88084100d72145207b3f8677
78bc87d00a7adbe1434c95b7f90df794
cf63bf7be69d9f8c474f3128e2561dca
9c6406226b61afe34194aae64eb91bea
66cf3631e2046ea2331763991fd73583
78bc87d00a7adbe1434c95b7f90df794
78bc87d00a7adbe1434c95b7f90df794
44bd995be349e958f1ca5d8b0d69af18
dd912e85bf0628884a52851874e6718a
ae3e35975fddc71d05e83dac7e5e8b84
fc9acaf6e3c1fd030215b3a06263dcef
68c7e67c8f7f466ada0c7834dfbf8c0d
4e8d73ff55e2f56040b1bfee1f662695
cb73ae2da632fa74fc902189bfeea81f
33660b9aee2846e76680f35b7095baaf
33660b9aee2846e76680f35b7095baaf
9fd71ff39eea7b84cb620a116254cb80
494c9ba8dfa6ea3021a4c82265ad2f85
3b4fff50e5a9ffe1913a807cc9093465
8d9904720b5252e427f1b02a4a6f4cda
7329a42825fd45a0869df27fc6e33d90
be4e809f063f4fea5bf3d350bbe41477
186e6f72c6

  3%|█▏                                           | 11397/455877 [00:00<00:17, 25610.82it/s]

35c6483a5049cf11b2076ef12f9d5de7
b3561c05a047693998c9c20b92b3494b
d0aa77c8629fef2ff5938eb34dba0175
ac82f0bbc35f29f28f028c498e72b5ca
317887f10604c3080a85452b53c38ba3
d16b8a0065e29fff9ea40cca07566926
ab210a5cbcd8372544244c5df2df7115
317887f10604c3080a85452b53c38ba3
6ea58e97e4bff02c720c0689988bb3ec
6ea58e97e4bff02c720c0689988bb3ec
6ea58e97e4bff02c720c0689988bb3ec
a69c5134248619b66af05471763308dc
a69c5134248619b66af05471763308dc
30d6049143eabaf08ce93303a0ff0450
30d6049143eabaf08ce93303a0ff0450
a0221cf01e4274143c1ebbc7dd4ca603
3c3ef3184b54fa08dc231797e3aca0c4
3c3ef3184b54fa08dc231797e3aca0c4
e337c7a92b497c7b11ec4a90ffc974cb
af943c72443dce729d1d355cc0d0ee2b
af943c72443dce729d1d355cc0d0ee2b
ef0c0e63cfc5ddad140831aa522f3ea3
3c3ef3184b54fa08dc231797e3aca0c4
ef0c0e63cfc5ddad140831aa522f3ea3
3c3ef3184b54fa08dc231797e3aca0c4
10898b64820e7280e6e2841d5faaf4c5
ef7d970121dba9d79a3da529045af415
ef0c0e63cfc5ddad140831aa522f3ea3
3c3ef3184b54fa08dc231797e3aca0c4
10898b64820e7280e6e2841d5faaf4c5
ef7d970121

  4%|█▉                                           | 20018/455877 [00:00<00:15, 27787.19it/s]

...

984181ce235af6cae80e5e34cf0ec7c6
20d20a4e6b5a103adb3c4774b5b474a4
19c1e24a4604af200bf6ef8cb4bead03
e89233c2b9df85628c533edc0bca0ae1
a1cbbc51faa4ecfec452071e87db588f
e89233c2b9df85628c533edc0bca0ae1
14647ae8db26c2b03603b21fa37a0372
ae6e5443f38f9a4a9930bab61bed31f9
a1cbbc51faa4ecfec452071e87db588f
5030c91c7faaf015c28748e608196993
ef5b9cd6bbd233ad8e875640204e6574
6ba7dcddfa9830a5c508476c585c4513
58a72b0f9befc7d4f6ce057f315a87e2
2fd7a548ab17b0304d3d7de3dfb8af91
17902f57440622f709af9f8d1ec964f9
ed37586bbd7c3ebcda785eb70227ad0d
846c6e967ecdcd4a31f758098b362ed0
3332a729e60758dd9742b43175d7eb27
e89233c2b9df85628c533edc0bca0ae1
641fa855d15acf2ce0fc991ef2c40b95
4e068130220056ee5840209af3031e08
464b6b3d1f050d6827fe45a659da6acc
4e068130220056ee5840209af3031e08
a1cbbc51faa4ecfec452071e87db588f
846c6e967ecdcd4a31f758098b362ed0
464b6b3d1f050d6827fe45a659da6acc
40f6972f0abc6abbf5eb8c22fc5ca6d8
641fa855d15acf2ce0fc991ef2c40b95
93ff372007375112620a42472891270a
302bf7d1d446021949e796db9fccb203
ddb797a20a

 13%|█████▉                                       | 60640/455877 [00:02<00:13, 28876.85it/s]

f5c3c3c6eb6acf2d6caa35ec1433a3d0
3294350833536c259e2613834c1735cd
3294350833536c259e2613834c1735cd
31c1946c418f1279f36170ac467632b8
189504cd5439d7ef7519a0d6980238de
14518209507612e5107eee3c5a7c747a
14518209507612e5107eee3c5a7c747a
cb267a2e1c0b8148ea02af981e9598dd
0fa6787d135df43e7d0e82462e9f1103
bdfa02e2407e57169775528eba26307f
2fee5364982dbc8c19c185c111742d11
c8e6d5867a06723580868063b2bb48d1
7b6b31ede273c9d9cf0500bdc5f79db5
87b6a5774491b721baa4710db68e3092
18a213c92eda8ab8d7218a58febb3255
fdd0545ffd6ddb9b9463c06a00efcd10
af5d41631bb72c3adc38b775bdf237ce
7430f9507565320d84421b8aae66de2c


 14%|██████                                       | 61898/455877 [00:02<00:14, 27530.19it/s]


KeyboardInterrupt: 